# Resume of Labels

In [9]:
import torch
import pandas as pd
import numpy as np
import os
import yaml

## Resume by filename

In [49]:
deep = "clase3"
files = os.listdir("data/"+ deep)
taxonomy_path = "taxonomy.yml"
print(files)

['audio1201.pt', 'audio1202.pt', 'audio1203.pt', 'audio1204.pt', 'audio1205.pt', 'audio1206.pt', 'audio1207.pt', 'audio1208.pt', 'audio1209.pt', 'audio1210.pt', 'audio1211.pt', 'audio1212.pt', 'audio1213.pt', 'audio1214.pt', 'audio1215.pt', 'audio1216.pt', 'audio1217.pt', 'audio1218.pt', 'audio1219.pt', 'audio1220.pt', 'audio1221.pt', 'audio1222.pt', 'audio1223.pt', 'audio1224.pt', 'audio1225.pt', 'audio1226.pt', 'audio1227.pt', 'audio1228.pt', 'audio1229.pt', 'audio1230.pt', 'audio1231.pt', 'audio1232.pt', 'audio1233.pt', 'audio1234.pt', 'audio1235.pt', 'audio1236.pt', 'audio1237.pt', 'audio1238.pt', 'audio1239.pt', 'audio1240.pt', 'audio1241.pt', 'audio1242.pt', 'audio1243.pt', 'audio1244.pt', 'audio1245.pt', 'audio1246.pt', 'audio1247.pt', 'audio1248.pt', 'audio1249.pt', 'audio1250.pt', 'audio1251.pt', 'audio1252.pt', 'audio1253.pt', 'audio1254.pt', 'audio1255.pt', 'audio1256.pt', 'audio1257.pt', 'audio1258.pt', 'audio1259.pt', 'audio1260.pt', 'audio1261.pt', 'audio1262.pt', 'audio1

In [50]:
with open(taxonomy_path) as taxpath:
    taxonomy = yaml.safe_load(taxpath)
    taxo = taxonomy['taxonomy']

In [51]:
df_dict = {"filename":[]}
for t in taxo.keys():
    df_dict[t] = []
for f in files:
    labels_f = torch.load(os.path.join("data",deep , f))
    sum_labels = np.sum(labels_f,axis=1)
    df_dict["filename"].append(f.split(".")[0])
    for i,t in enumerate(taxo.keys()):
        df_dict[t].append(int(sum_labels[i]))
df = pd.DataFrame(df_dict)
display(df)

,filename,Batrachyla_leptopus,Batrachyla_taeniata,Pleurodema_thaul,other_amphibians,Vanellus_chilensis,Theristicus_caudatus,Milvago_chimango,other_birds,dog,motor,alarm,siren,car_horn,anthropophony_others,rain_drip,rain_medium,rain_strong,blown,whistle
0,audio1201,0,0,0,0,0,0,0,0,5255,0,0,0,0,40,0,0,0,12920,0
1,audio1202,0,0,0,0,0,0,0,2638,463,291,0,0,0,0,0,0,0,12914,0
2,audio1203,0,0,0,0,0,0,633,123,1496,546,0,49,0,0,0,0,0,12901,0
3,audio1204,0,0,0,0,208,0,0,0,1899,0,0,0,0,0,0,0,0,11403,1564
4,audio1205,0,0,0,0,0,0,0,145,1727,0,0,0,0,0,0,0,0,12920,3959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,audio1496,0,0,0,0,445,3862,169,12920,3510,0,836,0,0,0,0,12920,918,12920,0
296,audio1497,0,0,0,0,0,425,0,12920,3592,1518,0,0,0,0,352,12893,0,12920,0
297,audio1498,0,0,1031,0,260,0,0,12909,872,595,1099,0,0,0,0,0,12920,12920,0
298,audio1499,0,0,0,0,1075,2452,179,12920,2219,5162,335,0,0,573,0,1026,0,12907,0


In [52]:
df.to_csv("resume_" + deep + ".csv",index = False)

## Resume by wetland

In [53]:
def num2meta(num,df):
    df_n = df[(num>=df.Start) & (num<=df.End)]
    day,month,year = df_n.Date.values[0].split(' de ')
    wetland = df_n.Wetland.values[0]
    if month == 'enero':
        m = '01'
    elif month == 'abril':
        m = '04'
    elif month == 'julio':
        m = '07'
    elif month == 'octubre':
        m = '10'
    return day,m,year,wetland

In [54]:
filenames = df.iloc[:,0].values
numfiles = [int(fn.replace("audio","")) for fn in filenames]
df_data = pd.read_csv("metadata.csv")

In [55]:
day = []
month = []
year = []
wetland= []

for n in numfiles:
    d,m,y,w = num2meta(n,df_data)
    day.append(d)
    month.append(m)
    year.append(y)
    wetland.append(w)
df["day"] = day
df["month"] = month 
df ["year"] = year
df["wetland"] = wetland  

In [56]:
df_wet = df.groupby(["wetland"]).sum()
df_date = df.groupby(["day","month","year"]).sum()

In [57]:
df_wet

,Batrachyla_leptopus,Batrachyla_taeniata,Pleurodema_thaul,other_amphibians,Vanellus_chilensis,Theristicus_caudatus,Milvago_chimango,other_birds,dog,motor,alarm,siren,car_horn,anthropophony_others,rain_drip,rain_medium,rain_strong,blown,whistle
wetland,,,,,,,,,,,,,,,,,,,
angachilla,342,702,172502,182,79840,83659,9238,592641,330874,178290,8420,2785,2728,114758,213814,162396,29253,762124,3166
el bosque,635,0,0,0,44453,267985,52241,460164,681666,286313,10786,30601,14579,76961,486265,329258,0,1550113,2753
miraflores,328,0,0,0,51384,112979,16353,335790,371490,240578,2884,32030,2316,63728,134108,127398,298551,1439887,163321


In [58]:
df_date

,,,Batrachyla_leptopus,Batrachyla_taeniata,Pleurodema_thaul,other_amphibians,Vanellus_chilensis,Theristicus_caudatus,Milvago_chimango,other_birds,dog,motor,alarm,siren,car_horn,anthropophony_others,rain_drip,rain_medium,rain_strong,blown,whistle
day,month,year,,,,,,,,,,,,,,,,,,,
20,07,2020,0,0,0,0,2734,64297,5376,92863,149726,85363,2020,5152,1887,7011,180827,103334,45345,600397,36711
21,07,2020,0,0,0,0,15559,48285,12236,49345,168311,111629,5867,12101,1324,1426,271222,103307,85442,555520,80273
22,07,2020,113,0,0,0,30969,97074,18072,173835,280351,143288,123,27037,9814,50518,83071,6673,30099,607109,12916
23,07,2020,850,0,0,0,19169,68391,17905,240607,259182,110652,1830,2465,1382,36190,0,153065,79981,607058,35454
24,07,2020,0,0,0,0,27406,102917,15005,239304,195586,75959,3830,15876,2488,45544,85253,90277,57684,619916,720
27,10,2020,0,505,40439,182,16399,35235,5685,211221,115964,84912,1491,1578,1694,24841,123035,105821,14722,310002,3166
28,10,2020,0,0,76761,0,43201,37274,3205,263653,138601,78296,3009,1207,717,88744,90427,13262,0,297108,0
29,10,2020,342,197,55302,0,20240,11150,348,117767,76309,15082,3920,0,317,1173,352,43313,14531,155014,0
